In [ ]:
import os
import numpy as np
import pandas as pd
import cv2

from tqdm import tqdm
from joblib import Parallel, delayed
print("hello")

In [ ]:
PAD = True
WIDTH = 512
HEIGHT = 512

In [ ]:
data_folder = "/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
train_folder = os.path.join(data_folder, 'train_images')
chain_names = os.listdir(train_folder)

print(os.listdir(data_folder))
print(len(chain_names))

In [ ]:
train_df = pd.DataFrame(columns={'image_id', 'hotel_id'})
for hotel_id in tqdm(chain_names):
    for image_id in os.listdir(os.path.join(train_folder, hotel_id)):
        train_df = train_df.append({'image_id': image_id, 'hotel_id': hotel_id}, ignore_index=True)

In [ ]:
len(train_df['hotel_id'].unique())

In [ ]:
class_info = []
for i in train_df['hotel_id'].unique():
    class_info.append({'class_len': len(train_df[train_df['hotel_id'] == i]), 'class_name': i})

In [ ]:
c = pd.DataFrame(class_info)
c = c.sort_values(['class_len'], ascending=False)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def fd(num, hotel_id, df):
    df = df.sample(num, replace=True)
    df['image_id'].apply(lambda x: aug_img(f'../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/{hotel_id}/{x}', hotel_id))

In [ ]:
def aug_img(image_id, hotel_id):
#     Original_Image = Image.open(f'../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/train_images/{hotel_id}/{image_id}')
    img = load_img(image_id)
    # convert to numpy array
    data = img_to_array(img)

    # expand dimension to one sample
    samples = expand_dims(data, 0)

    # create image data augmentation generator
    datagen = ImageDataGenerator(
        featurewise_center=True,
        rotation_range=(0-30),
        width_shift_range=0.2,                
        height_shift_range=0.2,
        brightness_range=[0.5,1.5],
        shear_range=0.2, 
        zoom_range=0.2,
        channel_shift_range=0.2,
        horizontal_flip=True, 
        vertical_flip=True,
        fill_mode='nearest',
    )
    # prepare iterator
    it = datagen.flow(samples, batch_size=1, save_to_dir='.', save_prefix=f'{hotel_id}-aug-',
                                            save_format='jpg')
    
    # generate samples and plot
    batch = it.next()

In [ ]:
# random.sample(the_list, 50)
# train_df
global count
count = 0

global below
below = []
global after
after = []
def cl(x):
    global count
    count = count + 1
    print(count)
    if x['class_len'] > 40:
        class_imgs = train_df[train_df['hotel_id'] == x['class_name']]
        global below
        below.extend(class_imgs.sample(40).to_dict('records'))
    else:
        class_imgs = train_df[train_df['hotel_id'] == x['class_name']]
        fd(40-len(class_imgs), x['class_name'], class_imgs)
# c.apply(lambda x: cl(x), axis=1)

In [ ]:
%%time
dfs_proc =Parallel(n_jobs=4, prefer='threads')(delayed(cl)(i) for v, i in c.iterrows())

In [ ]:
import glob
global top
top = []
def get_csv(x):
    name = x['class_name']
    jpgFilenamesList = glob.glob(f'{name}*.jpg')
    for i in jpgFilenamesList:
        top.append({'hotel_id': name, 'image_id': i})

c.apply(lambda x: get_csv(x), axis=1)
top.extend(below)

In [ ]:
def pad_image(img):
    w, h, c = np.shape(img)
    if w > h:
        pad = int((w - h) / 2)
        img = cv2.copyMakeBorder(img, 0, 0, pad, pad, cv2.BORDER_CONSTANT, value=0)
    else:
        pad = int((h - w) / 2)
        img = cv2.copyMakeBorder(img, pad, pad, 0, 0, cv2.BORDER_CONSTANT, value=0)
        
    return img


def open_and_preprocess_image(img):
    print(os.path.join('.', img))
    img = cv2.imread(os.path.join('.', img))
    if PAD:
        img = pad_image(img)
    
    return cv2.resize(img, (WIDTH, HEIGHT))


def save_image(image_name, img):
    cv2.imwrite(image_name, img)
    
    
def process_chain(img):
#     chain_folder = os.path.join(data_folder, chain_name)
    name = img['image_id']
    img = open_and_preprocess_image(img['image_id'])
    save_image(name, img)

In [ ]:
%%time
dfs_proc = Parallel(n_jobs=4, prefer='threads')(delayed(process_chain)(top[i]) for i in range(0, len(top)))

In [ ]:
!cd /kaggle/working/ & zip -jqr images.zip .
!find . -name "*.jpg" -delete
top.to_csv('train.csv', index=False)